In [38]:
import pandas as pd
import numpy as np

In [39]:
# Behavioral measures
row_names =  ['Visual Episodic Memory','Cognitive Flexibility','Inhibition','Fluid Intelligence','Reading','Vocabulary','Processing Speed','Spatial Orientation','Sustained Attention_Sens','Sustained Attention_Spec','Verbal Episodic Memory','Working Memory','Cognitive Status',
                 'Sleep Quality','Walking Endurance','Manual Dexterity','Grip Strength','Odor Identification','Pain Interference Survey','Taste Intensity','Contrast Sensitivity','Emotional Face Matching','Arithmetic',
                 'Story Comprehension','Relational Processing','Social Cognition_Random','Social Cognition_Interaction','Working Memory (N-back)','Agreeableness','Openness','Conscientiousness','Neuroticism','Extraversion','Emot. Recog. _ Total','Emot. Recog. _ Angry','Emot. Recog. _Fear',
                 'Emot. Recog. _ Happy','Emot. Recog. _ Neutral','Emot. Recog. _ Sad','Anger _ Affect','Anger _ Hostility','Anger _ Aggression','Fear _ Affect','Fear _ Somatic Arousal','Sadness','Life Satisfaction','Meaning & Purpose','Positive Affect','Friendship',
                 'Loneliness','Perceived Hostility','Perceived Rejection','Emotional Support','Instrument Support','Perceived Stress','Self-Efficacy','Delay Discounting','Walking Speed']

In [40]:
# pca components
fold1 = np.array([[-0.07292692, -0.0765595 , -0.03287499, -0.06943219, -0.03928723,
        -0.04714184, -0.05729954, -0.03958908, -0.03755193, -0.04589162,
        -0.06165279, -0.06473069, -0.06087831,  0.13553084, -0.05099981,
        -0.0407317 , -0.01111563,  0.00162416,  0.12154608,  0.00786219,
         0.01582667, -0.00238274, -0.05710059, -0.06985408, -0.06725868,
        -0.02765289, -0.02436173, -0.08911347, -0.13780858, -0.00061138,
        -0.14268517,  0.25262373, -0.17221542, -0.01792332, -0.01005983,
        -0.00434175, -0.02529953, -0.01439137, -0.00620345,  0.23272515,
         0.22200757,  0.12365637,  0.2189362 ,  0.12190866,  0.25643667,
        -0.22622002, -0.2019512 , -0.22109689, -0.20139072,  0.24828433,
         0.19371665,  0.23558597, -0.21667018, -0.15089202,  0.26951674,
        -0.19577335, -0.02563544, -0.02047543],
       [-0.14294803, -0.1878532 , -0.15953833, -0.27611985, -0.31230615,
        -0.29533491, -0.15722505, -0.24347498, -0.09417382, -0.0947212 ,
        -0.1394145 , -0.21236946, -0.12118481,  0.01796112, -0.18536543,
        -0.10732254, -0.05168087, -0.0389389 ,  0.01333896,  0.08291609,
         0.00985983, -0.05775246, -0.16691307, -0.23792301, -0.26444743,
        -0.01412226, -0.06800331, -0.26629739,  0.00454958, -0.14120056,
         0.1039155 , -0.03499467,  0.07511985, -0.16446073, -0.08093686,
        -0.09984928, -0.0132405 , -0.07018986, -0.11648935, -0.09910599,
        -0.0220339 ,  0.00971392, -0.06842147, -0.06618892, -0.10219014,
        -0.00136835,  0.10402452,  0.08733065,  0.05756203, -0.1075926 ,
        -0.04765326, -0.02134611,  0.05218515,  0.00412959, -0.036199  ,
         0.01816038, -0.11219727, -0.05425558],
       [-0.03842861, -0.06226607,  0.00306107,  0.06505846,  0.11485702,
         0.12824041, -0.05429833,  0.06164222,  0.06014299, -0.02665472,
        -0.03308435,  0.00387007,  0.02368572, -0.03474766,  0.12591466,
        -0.03017886,  0.162138  ,  0.00455116,  0.04151861, -0.04883489,
         0.00624464, -0.03972412,  0.00110599,  0.07204478,  0.0450331 ,
        -0.15033498,  0.13644465,  0.07160628, -0.08914921,  0.12554129,
        -0.123453  , -0.01388728,  0.06846496, -0.56777977, -0.29731814,
        -0.37402104, -0.12315773, -0.26827921, -0.31099763,  0.08572365,
         0.04017093,  0.0533898 ,  0.02085692,  0.05998374,  0.0117437 ,
         0.07342241,  0.04492697,  0.0309162 ,  0.03166325, -0.02742196,
         0.11138592,  0.02418324,  0.06308063,  0.07340412,  0.00649056,
         0.0824868 ,  0.10325625,  0.01110392]])
fold2 = np.array([[-0.07714396, -0.05808676, -0.0382643 , -0.06246209, -0.06074776,
        -0.05649977, -0.04072571, -0.0362369 , -0.0321788 , -0.04043211,
        -0.04632621, -0.05097181, -0.07387626,  0.1258806 , -0.04983827,
        -0.05825661,  0.00658725, -0.01332447,  0.12802978,  0.01253274,
        -0.01811954, -0.00539139, -0.04609602, -0.0684301 , -0.08599972,
        -0.02292761, -0.00973017, -0.08666934, -0.16148634, -0.02084058,
        -0.12991618,  0.25112915, -0.14990726, -0.02207536,  0.00231898,
        -0.01440047, -0.02421365, -0.01770961, -0.01151578,  0.22855592,
         0.22321835,  0.11288289,  0.21875089,  0.12522759,  0.25346174,
        -0.2342914 , -0.20177068, -0.21125643, -0.20343853,  0.25099527,
         0.18843051,  0.23842535, -0.22605424, -0.16788307,  0.27048841,
        -0.1887211 , -0.04911567, -0.0265168 ],
       [-0.16570082, -0.1920113 , -0.15255405, -0.27005924, -0.29980227,
        -0.28292635, -0.15153727, -0.23976719, -0.06782312, -0.11545145,
        -0.15291967, -0.20440227, -0.14153657,  0.03570741, -0.18400953,
        -0.12524682, -0.03276977, -0.03092847, -0.00196953,  0.0896322 ,
        -0.05331455, -0.07542942, -0.17157372, -0.20743235, -0.25069042,
        -0.00085172, -0.06092769, -0.26313198, -0.00136209, -0.14747485,
         0.13474421, -0.04775615,  0.07626096, -0.18059657, -0.10874168,
        -0.09388955, -0.01960551, -0.07519968, -0.11591472, -0.0738363 ,
        -0.04609775,  0.01445239, -0.09300892, -0.07500228, -0.10270283,
        -0.00427239,  0.08170754,  0.06570174,  0.06139978, -0.10534137,
        -0.04302283, -0.04743377,  0.03839584,  0.02780087, -0.04278957,
         0.02759073, -0.13916876, -0.02737864],
       [-0.07419545,  0.07975106,  0.08394962,  0.04954836,  0.05325625,
         0.03183428,  0.0936256 ,  0.09549692,  0.09662443, -0.08965069,
        -0.12771589,  0.09032326, -0.08772239, -0.0113826 ,  0.1410563 ,
        -0.16945493,  0.33081222, -0.14280451,  0.05971493, -0.05349683,
         0.0814653 ,  0.08222513,  0.03885218,  0.19427161,  0.03506461,
        -0.03585027,  0.02133251,  0.11902385, -0.2098065 , -0.00880841,
        -0.08656741, -0.05986638,  0.06738776, -0.44607362, -0.26123155,
        -0.28528685, -0.10322985, -0.1296098 , -0.28882094,  0.08639758,
         0.0757754 ,  0.21418398, -0.06371779,  0.00724862, -0.04074502,
         0.03731569,  0.01751528,  0.07071599,  0.06509142, -0.03148464,
         0.15002701,  0.04768765,  0.0123995 ,  0.07471622, -0.02185292,
         0.12157704,  0.04515772, -0.09368149]])
fold3 = np.array([[-0.06572031, -0.06717898, -0.04095569, -0.06255087, -0.05361708,
        -0.05624203, -0.04674011, -0.04385431, -0.01772826, -0.05606641,
        -0.05413906, -0.05807064, -0.06011339,  0.12186651, -0.05786196,
        -0.06704239, -0.00793193, -0.0057066 ,  0.12109979,  0.02239942,
         0.01417742, -0.01104129, -0.05010451, -0.07042665, -0.06104543,
        -0.03651955, -0.02690103, -0.07395565, -0.15321057, -0.00560344,
        -0.14290972,  0.24783833, -0.15788601, -0.06177319, -0.01288088,
        -0.03191859, -0.03642704, -0.03549278, -0.04576897,  0.22075236,
         0.22433661,  0.13179392,  0.21600501,  0.12379033,  0.24921957,
        -0.23192283, -0.20492023, -0.21617036, -0.20296919,  0.24799034,
         0.18716872,  0.23705971, -0.22057813, -0.16826213,  0.26318287,
        -0.19459721, -0.04861959, -0.04089378],
       [-0.18179141, -0.1958571 , -0.16084227, -0.26506067, -0.2975356 ,
        -0.28129982, -0.15277029, -0.23376233, -0.11662055, -0.11698881,
        -0.15457806, -0.2139183 , -0.15032468,  0.00206574, -0.17690489,
        -0.12003252, -0.05240742, -0.01719241, -0.03420491,  0.09777404,
         0.00531395, -0.08362912, -0.16014964, -0.20592639, -0.25566707,
        -0.03013803, -0.08970372, -0.27038872,  0.01552211, -0.11744839,
         0.10137904, -0.0449263 ,  0.08443069, -0.16892993, -0.09676032,
        -0.11020244, -0.02713436, -0.05966632, -0.10156746, -0.09061131,
        -0.04477961,  0.0030265 , -0.08996653, -0.11427499, -0.0871514 ,
         0.00403434,  0.08405857,  0.07021866,  0.06624935, -0.09001641,
        -0.04820459, -0.04635905,  0.04631662,  0.02064726, -0.04815047,
         0.02007537, -0.11659706, -0.01854016],
       [-0.00867386,  0.05099198,  0.09971258,  0.01892699, -0.01220701,
         0.02366   ,  0.08013235,  0.0337894 ,  0.01468062, -0.03796277,
        -0.07618478,  0.08663805,  0.01080911, -0.10364316,  0.12066299,
        -0.11385262,  0.33546402, -0.06613539, -0.06735859, -0.11898755,
         0.02036415,  0.03293315,  0.13560308,  0.19707247,  0.01287301,
        -0.01537081,  0.04852594,  0.12780504, -0.23378683, -0.05325686,
        -0.06792078, -0.08923599, -0.03343578, -0.46814277, -0.25065459,
        -0.35728845, -0.01938218, -0.12787361, -0.29493129, -0.01862908,
         0.07016605,  0.21105385, -0.15068579, -0.07436908, -0.0986271 ,
         0.04791742, -0.05013453,  0.03927491, -0.05606041,  0.00294457,
         0.12204704,  0.01548768, -0.06534156,  0.03818257, -0.07688846,
         0.05778806, -0.00120986, -0.00684355]])

In [41]:
# Inspect each fold seperately
df = {}
df[0] = pd.DataFrame(fold1.T,index=row_names)
df[1] = pd.DataFrame(fold2.T,index=row_names)
df[2] = pd.DataFrame(fold3.T,index=row_names)

for i in range(3):
    print(f"fold{i}:\n")
    for j in range(3):
        print(f"pc{j}:\n",df[i][j].nlargest(10))
    print("\n")

fold0:

pc0:
 Perceived Stress       0.269517
Sadness                0.256437
Neuroticism            0.252624
Loneliness             0.248284
Perceived Rejection    0.235586
Anger _ Affect         0.232725
Anger _ Hostility      0.222008
Fear _ Affect          0.218936
Perceived Hostility    0.193717
Sleep Quality          0.135531
Name: 0, dtype: float64
pc1:
 Meaning & Purpose           0.104025
Conscientiousness           0.103915
Positive Affect             0.087331
Taste Intensity             0.082916
Extraversion                0.075120
Friendship                  0.057562
Emotional Support           0.052185
Self-Efficacy               0.018160
Sleep Quality               0.017961
Pain Interference Survey    0.013339
Name: 1, dtype: float64
pc2:
 Grip Strength                   0.162138
Social Cognition_Interaction    0.136445
Vocabulary                      0.128240
Walking Endurance               0.125915
Openness                        0.125541
Reading                        

In [42]:
# Inspect the loadings by averaging across folds 
df_stack = np.stack([fold1,fold2,fold3])
df_mean = pd.DataFrame(np.mean(df_stack,axis=0).T,index=row_names)
pc0_top_10 = df_mean[0].nlargest(10)
pc1_top_10 = df_mean[1].nlargest(10)
pc2_top_10 = df_mean[2].nlargest(10)
print("pca0:\n",pc0_top_10)
print("pca1:\n",pc1_top_10)
print("pca2:\n",pc2_top_10)

pca0:
 Perceived Stress       0.267729
Sadness                0.253039
Neuroticism            0.250530
Loneliness             0.249090
Perceived Rejection    0.237024
Anger _ Affect         0.227344
Anger _ Hostility      0.223188
Fear _ Affect          0.217897
Perceived Hostility    0.189772
Sleep Quality          0.127759
Name: 0, dtype: float64
pca1:
 Conscientiousness     0.113346
Taste Intensity       0.090107
Meaning & Purpose     0.089930
Extraversion          0.078604
Positive Affect       0.074417
Friendship            0.061737
Emotional Support     0.045633
Self-Efficacy         0.021942
Sleep Quality         0.018578
Instrument Support    0.017526
Name: 1, dtype: float64
pca2:
 Grip Strength                   0.276138
Anger _ Aggression              0.159543
Story Comprehension             0.154463
Walking Endurance               0.129211
Perceived Hostility             0.127820
Working Memory (N-back)         0.106145
Self-Efficacy                   0.087284
Social Cogniti